In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix,f1_score,roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import joblib

# KNN (Classification)
def classification_KNN(trainX, trainY, testX, testY, decision_boundary=0.5, k=3):
    '''
    @Input
        x_train : type : array
        y_train : type : array
        x_test : type : array
        k : type : int
    @Output
        auc_score : type : float64
        fpr : type : array
        fpr : type : array
    '''

    if isinstance(trainX, pd.DataFrame):
        trainX = trainX.values
    if isinstance(testX, pd.DataFrame):
        testX = testX.values
    if isinstance(trainY, pd.DataFrame):
        trainY = trainY.values
    if isinstance(testY, pd.DataFrame):
        testY = testY.values

    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(trainX, trainY)
    anomalyScore = model.predict_proba(testX)
    
    pred = np.array([], dtype=np.int32)
    for i in range(len(anomalyScore)):
        if (anomalyScore[i][1] > decision_boundary) == True:
            pred = np.append(pred, 1)
        else:
            pred = np.append(pred, 0)
            
    # # KNN 모델 Pickle 파일로 저장
    saveModel = joblib.dump(model, 'KNN.pkl')

    fpr_array = np.array([], dtype=np.int32)
    tpr_array = np.array([], dtype=np.int32)

    for i in range(1, 100):
        predd = np.array([], dtype=np.int32)
        for j in range(len(anomalyScore)):
            decision_boundary = i / 100
            if (anomalyScore[j][1] > decision_boundary) == True:
                predd = np.append(predd, 1)
            else:
                predd = np.append(predd, 0)

        cofMatt = confusion_matrix(testY, predd, labels=[1, 0])  # 1: 불량, 0: 정상

        ##confusion matrix의 각 요소
        auc_tpr = cofMatt[1, 1] / (cofMatt[1, 1] + cofMatt[1, 0])
        auc_fpr = cofMatt[0, 1] / (cofMatt[0, 0] + cofMatt[0, 1])
        fpr_array = np.append(fpr_array, auc_fpr)
        tpr_array = np.append(tpr_array, auc_tpr)

    auc = roc_auc_score(testY, pred)
    cofMat = confusion_matrix(testY, pred, labels=[1, 0])  # 1: 불량, 0: 정상
    tpr = cofMat[1, 1] / (cofMat[1, 1] + cofMat[1, 0])
    fpr = cofMat[0, 1] / (cofMat[0, 0] + cofMat[0, 1])

    anomalyScore = np.round(anomalyScore[:, 1], 3)

    return {"AnomalyScore" : anomalyScore, "Prediction":pred, "Aucscore": str(auc), "Fpr": fpr, "Tpr": tpr, "Fprarray":fpr_array, "Tprarray":tpr_array}

def classification_model_load_KN(model_wd, testX, decision_boundary):
    model_wd = 'KNN.pkl'
    model = joblib.load(model_wd)
    
    anomalyScore = model.predict_proba(testX)
    pred = np.array([], dtype=np.int32)
    for i in range(len(anomalyScore)):
        if (anomalyScore[i][1] > decision_boundary) == True:
            pred = np.append(pred, 1)
        else:
            pred = np.append(pred, 0)
    

    return {"AnomalyScore":np.round(anomalyScore[:,1],3), "Prediction": pred}

In [ ]:
df = pd.read_csv('/workspace/xnsolution/data/1_Iris_virginica.csv', encoding='euc-kr')

df['label'][np.where(df['label'] == 'outlier')[0]] = 1
df['label'][np.where(df['label'] == 'target')[0]] = 0

X = df.iloc[:,0:4]
y = df.iloc[:,4]
y= y.astype('category')

trainX, testX, trainY, testY = train_test_split(X,y, test_size=0.3)

model = classification_KNN(trainX, trainY, testX, testY, decision_boundary=0.5)
load_model = classification_model_load_KN('KNN.pkl', testX, decision_boundary=0.5)